In [ ]:
import sys
import argparse

import torch
import pytorch_lightning as pl

sys.path.append('../')
from datasets.dist import Gaussian, GMM
from model.cnf import CNF, parse_verlet_cnf_args, VerletCNF
%load_ext autoreload
%autoreload 2

## 2D-CNF

In [ ]:
# Initialize source and target
args = argparse.Namespace(num_hidden_units = 16, 
                          num_timesteps = 25,
                          source = 'gaussian',
                          target = 'gmm',
                          target_nmode = 2)       

# Initialize flow
cnf = CNF(args)

# Train
trainer = pl.Trainer(min_epochs=15, max_epochs=20)
trainer.fit(cnf)

In [ ]:
cnf.graph_marginals(100000)

In [ ]:
cnf.graph_backwards_marginals(100000)

In [ ]:
trainer.save_checkpoint(f'../workdir/{NAME_HERE].ckpt')

## Verlet-CNF

In [ ]:
# Initialize source and target
args = parse_verlet_cnf_args([])  

# Initialize flow
cnf = VerletCNF(args)

# Train
trainer = pl.Trainer(min_epochs=5, max_epochs=5)
trainer.fit(cnf)

In [ ]:
cnf.target.q_dist.graph(10000)

In [ ]:
cnf.graph_marginals(100000)

In [ ]:
cnf.graph_backwards_marginals(10000)